In [1]:
import csv
import os
import re
import sqlite3
import uuid
from datetime import datetime, timedelta
from typing import Any, Callable

from camelot import read_pdf
from dotenv import load_dotenv
from httpx import HTTPStatusError, get
from pandas import (
    Categorical,
    DataFrame,
    Series,
    concat,
    isna,
    read_csv,
    set_option,
    to_datetime,
    to_numeric,
)
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
from sqlalchemy.types import TIMESTAMP
from tabulate import tabulate

load_dotenv()  # i don't use dotenv, but in case you didn't config vscode previously

set_option("display.max_columns", None)

c:\Users\VDUART10\Desktop\Tests\test_indicium\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extraindo Schema de dados do dicionario de dados

Para este caso em especifico eu não fiz nenhuma estrutura de processamento de texto por que o intuito era extrair a informação de forma rapida.

Poderia-se criar uma pipeline de extração de conteudo cru com uma lib como `pdfplumber` extraindo texto bruto e iterando sobre coletando nome, tipo e info das colunas + metadados opcionais, mas optei por agilidade na solução. Isso aqui teoricamente é pra ser feito uma vez só, então não gastei muitos recursos encima, como faria numa solução real

In [ ]:
def df_to_markdown_table(df: DataFrame) -> str:
    """
    Convert a pandas DataFrame to a clean markdown table format
    """
    df = df.dropna(axis=1, how="all").fillna("")

    # Clean data by removing excessive whitespace
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str).str.strip()

    return tabulate(df, headers="keys", tablefmt="pipe", showindex=False)


def is_continuation_row(row: Series) -> bool:
    """
    Check if a row appears to be a continuation from a previous page.

    A continuation row is identified when:
    - Columns 0-2 are empty/null
    - Columns 3-5 contain data

    Args:
        row (Series): A pandas Series representing a single row from a dataframe

    Returns:
        bool: True if the row appears to be a continuation row, False otherwise

    Note:
        Requires at least 5 columns in the row to perform the check.
        Returns False if the row has fewer than 5 columns.
    """
    if len(row) < 5:
        return False

    return all(  # Check if first 3 columns are empty
        isna(row.iloc[j]) or str(row.iloc[j]).strip() == "" for j in range(3)
    ) and any(  # Check if columns 3-5 have data
        not isna(row.iloc[j]) and str(row.iloc[j]).strip() != "" for j in range(2, 5)
    )


def merge_continuation_row(target_df: DataFrame, continuation_row: Series) -> None:
    """
    Merge a continuation row with the last row of the target dataframe.

    The function modifies the target dataframe in-place by:
    - Filling empty cells in the last row with data from the continuation row
    - Concatenating content when both cells contain data

    Args:
        target_df (DataFrame): The dataframe whose last row will be updated
        continuation_row (Series): The row containing continuation data

    Returns:
        None: Modifies target_df in-place

    Note:
        If target_df is empty, the function returns without making changes.
        Only processes columns that exist in both the target dataframe and continuation row.
    """
    if len(target_df) == 0:
        return

    last_row_idx = target_df.index[-1]

    for col_idx in range(len(continuation_row)):
        if col_idx >= len(target_df.columns):
            break

        col_name = target_df.columns[col_idx]
        continuation_value = continuation_row.iloc[col_idx]

        # Skip if continuation cell is empty
        if isna(continuation_value) or str(continuation_value).strip() == "":
            continue

        existing_value = target_df.loc[last_row_idx, col_name]

        # If target cell is empty, use continuation value
        if isna(existing_value) or str(existing_value).strip() == "":
            target_df.loc[last_row_idx, col_name] = continuation_value
        else:
            # Both have content, concatenate with space
            existing = str(existing_value).strip()
            new_content = str(continuation_value).strip()
            target_df.loc[last_row_idx, col_name] = f"{existing} {new_content}"


def process_single_dataframe(df: DataFrame, previous_df: DataFrame | None) -> DataFrame:
    """
    Process a single dataframe, checking for continuation rows and handling merging.

    If the first row of the dataframe is identified as a continuation row and there's
    a previous dataframe available, the continuation row is merged with the last row
    of the previous dataframe and removed from the current dataframe.

    Args:
        df (DataFrame): The dataframe to process
        previous_df (Optional[DataFrame]): The previous dataframe for potential merging.
                                            None if this is the first dataframe.

    Returns:
        DataFrame: A copy of the processed dataframe with continuation rows removed
                     if they were merged with the previous dataframe

    Note:
        If df is empty, returns an empty copy.
        The previous_df is modified in-place if a merge occurs.
    """
    if len(df) == 0:
        return df.copy()

    current_df = df.copy()
    first_row = current_df.iloc[0]

    # Check if first row is a continuation row and we have a previous dataframe
    if is_continuation_row(first_row) and previous_df is not None:
        # Merge with previous dataframe
        merge_continuation_row(previous_df, first_row)
        # Remove the merged row from current dataframe
        current_df = current_df.iloc[1:].copy()

    return DataFrame(current_df)


def merge_split_rows(dfs: list[DataFrame]) -> list[DataFrame]:
    """
    Merge rows that are split across PDF pages in a list of dataframes.

    This function processes a list of dataframes (typically representing tables from
    different PDF pages) and merges rows that were split across page boundaries.
    Split rows are identified using the continuation row detection logic.

    Args:
        dfs (List[DataFrame]): List of dataframes to process, typically one per PDF page

    Returns:
        List[DataFrame]: List of processed dataframes with split rows merged.
                           Empty dataframes are excluded from the result.

    Note:
        The function processes dataframes sequentially, so continuation rows are only
        merged with the immediately preceding dataframe in the list.
        Original dataframes are not modified; copies are created for processing.

    Example:
        >>> dfs = [df_page1, df_page2, df_page3]
        >>> merged_dfs = merge_split_rows(dfs)
        >>> final_df = concat(merged_dfs, ignore_index=True)
    """
    if not dfs:
        return []

    processed_dfs: list[DataFrame] = []

    for i, df in enumerate(dfs):
        processed_df = process_single_dataframe(
            df, processed_dfs[-1] if processed_dfs else None
        )
        if len(processed_df) > 0:
            processed_dfs.append(processed_df)

    return processed_dfs


def clean_markdown_text(text):
    """
    Clean and enhance the markdown text for better LLM processing
    """
    cleaned_lines = []

    for line in text.split("\n"):
        # Trim whitespace
        line = line.strip()

        # Skip completely empty lines except single newlines
        if line != "" or (cleaned_lines and cleaned_lines[-1] != ""):
            cleaned_lines.append(line)

    return "\n".join(cleaned_lines)

In [ ]:
try:
    content = read_pdf(
        "../docs/dicionario-de-dados-2019-a-2025.pdf", pages="all", line_scale=40
    )
    for df in (dfs := [table.df for table in content][1:]):
        df.columns = dfs[0].iloc[0]  # Set the first row as column names

    dfs[0].drop(index=0, inplace=True)
    df_list = [
        "## Concatenated Table\n\n"
        + df_to_markdown_table(
            all_df := concat(merge_split_rows(dfs), ignore_index=True)
        )
        + "\n"
    ]
    all_df["Tipo"] = all_df["Tipo"].str.replace("\n", "")
    all_df.to_csv("../docs/data_dict.csv", index=False)
    print("Successfully extracted markdown tables from the PDF.")
    with open("../docs/data_dict.md", "w") as f:
        f.write(clean_markdown_text("\n".join(df_list)))
    print("Markdown conversion completed successfully!")
except Exception as e:
    print(f"Error extracting tables: {e}")
    raise e

In [ ]:
with (
    open("../docs/data_dict.csv", newline="", encoding="utf-8") as fin,
    open("../docs/data_dict_formated.csv", "w", newline="", encoding="utf-8") as fout,
):
    reader = csv.reader(fin)
    writer = csv.writer(fout)
    # Cabeçalho unificado
    writer.writerow(
        [
            "field_name",
            "data_type",
            "category",
            "description",
            "characteristics",
            "db_field",
        ]
    )
    for row in reader:
        if not any(cell.strip() for cell in row):
            continue
        # Junta colunas que ficaram “quebradas” em descrição
        # Supondo que DBF (última coluna) nunca contenha vírgula
        *middle, dbf = row
        # As três primeiras colunas fixas
        raw_name, raw_type, raw_cat = middle[:3]
        rest = middle[3:]
        # Descrição + características podem estar mescladas
        # Separa descrição de características por padrão de texto
        if '"' in (text := " ".join(cell.strip() for cell in rest if cell.strip())):
            desc, _, chars = text.partition("Campo")
            chars = "Campo" + chars if chars else ""
        else:
            desc, chars = text, ""
        writer.writerow(
            [
                re.sub(r"[^a-z0-9]+", "_", raw_name.strip().lower()).strip("_"),
                raw_type.replace("\n", " ").strip(),
                raw_cat.replace("\n", " ").strip(),
                desc.strip().replace("\n", " "),
                chars.strip().replace("\n", " "),
                dbf.strip(),
            ]
        )

## Extraindo bases de dados e dividindo entre ambientes
- Esse processo aqui eu jogaria esses Data Lake pra ser acessivel ao time.
- Vamos fingir q o diretorio data é nosso Lake.
- Com um sqlite tambem simulando um postgres, se sobrar tempo eu troco por um Postgres num docker

In [4]:
MIXED_TYPE_COLS = [  # tá hardcoded, mas é o que tenho por agora
    14,
    17,
    66,
    92,
    94,
    106,
    108,
    115,
    117,
    118,
    119,
    121,
    122,
    123,
    129,
    144,
    146,
    149,
    155,
    159,
    160,
    161,
    162,
    163,
    164,
    165,
    166,
    167,
    168,
    169,
    170,
    171,
    172,
    173,
    174,
    175,
    176,
    181,
    185,
    186,
    188,
    189,
]


In [2]:
def get_column_type_mapping(data_dict_df: DataFrame) -> dict[str, str]:
    """
    Enhanced type mapping with better pattern matching and validation.
    """
    type_mapping = {}

    for _, row in data_dict_df.iterrows():
        col_name = row["DBF"]
        col_type = row["Tipo"]

        if isna(col_name) or isna(col_type):  # type: ignore
            continue

        col_type_clean = str(col_type).strip().replace(" ", "")

        # Enhanced pattern matching
        if re.search(r"Date|Data", col_type_clean, re.IGNORECASE):
            type_mapping[col_name] = "datetime64[ns]"
        elif re.search(r"Number\(\d+\)", col_type_clean, re.IGNORECASE):
            type_mapping[col_name] = "float64"
        elif re.search(r"Varchar2?\(1\)", col_type_clean, re.IGNORECASE):
            # Single character fields are good candidates for categories
            type_mapping[col_name] = "category"
        elif re.search(r"Varchar2?\(\d+\)", col_type_clean, re.IGNORECASE):
            type_mapping[col_name] = "string"
        else:  # Default fallback for malformed or unrecognized types
            type_mapping[col_name] = "string"

    return type_mapping


def create_converters(dtype_mapping: dict[str, str]) -> dict[str, Callable]:
    """
    Create converter functions with proper scope handling.
    """
    converters = {}

    for col_name, dtype in dtype_mapping.items():
        if dtype == "datetime64[ns]":
            converters[col_name] = lambda x: to_datetime(
                x, format="%d/%m/%Y", errors="coerce"
            )
        elif dtype == "float64":
            converters[col_name] = lambda x: to_numeric(x, errors="coerce")
        elif dtype == "category":
            converters[col_name] = lambda x: Categorical(x)
        elif dtype == "string":
            converters[col_name] = lambda x: x.astype("string")

    return converters


def apply_dtype_conversions(df: DataFrame, dtype_mapping: dict[str, str]) -> DataFrame:
    """
    Apply data type conversions to DataFrame.
    """
    df_converted = df.copy()

    for col_name, target_dtype in dtype_mapping.items():
        if col_name not in df_converted.columns:
            continue

        try:
            if target_dtype == "datetime64[ns]":
                df_converted[col_name] = to_datetime(
                    df_converted[col_name], format="%d/%m/%Y", errors="coerce"
                )
            elif target_dtype == "float64":
                df_converted[col_name] = to_numeric(
                    df_converted[col_name], errors="coerce"
                )
            elif target_dtype == "category":
                df_converted[col_name] = df_converted[col_name].astype("category")
            elif target_dtype == "string":
                df_converted[col_name] = df_converted[col_name].astype("string")
        except Exception as e:
            print(
                f"Warning: Could not convert column {col_name} to {target_dtype}: {e}"
            )

    return df_converted


def optimize_for_target_format(df: DataFrame, target_format: str) -> DataFrame:
    """
    Optimize DataFrame types for Parquet or SQLite.
    - parquet: converts suitable object cols to 'category'
    - sqlite: ensures categories & mixed cols are strings
    """
    df_optimized = df.copy()
    target_format = target_format.lower()

    # Resolve column names from indices (skip if index > ncols)
    mixed_cols = [
        df_optimized.columns[i]
        for i in MIXED_TYPE_COLS
        if i < len(df_optimized.columns)
    ]
    if target_format == "parquet":
        for col in mixed_cols:
            if df_optimized[col].dtype == "object":
                # Try to coerce to numeric if possible
                coerced = to_numeric(df_optimized[col], errors="coerce")
                if coerced.notna().mean() > 0.9:  # >90% convertible
                    df_optimized[col] = coerced
                else:  # Use category if low cardinality
                    if df_optimized[col].nunique() / len(df_optimized) < 0.5:
                        df_optimized[col] = df_optimized[col].astype("category")

        # Handle remaining object columns (same as your original)
        for col in df_optimized.select_dtypes(include=["object"]).columns:
            if df_optimized[col].nunique() / len(df_optimized) < 0.5:
                df_optimized[col] = df_optimized[col].astype("category")

    elif target_format == "sqlite":
        # SQLite doesn’t support category → convert to string
        for col in df_optimized.select_dtypes(include=["category"]).columns:
            df_optimized[col] = df_optimized[col].astype("string")
        # Force mixed-type columns to string (to avoid type conflicts)
        for col in mixed_cols:
            df_optimized[col] = df_optimized[col].astype("string")

    return df_optimized

In [ ]:
dtype_mapping = get_column_type_mapping(all_df)

DFS: list[DataFrame] = []
for y in range(19, 25):
    print(f"Retriving year 20{y} dataset... ", end="")
    DFS.append(
        df := apply_dtype_conversions(
            read_csv(
                f"https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/20{y}/INFLUD{y}-26-06-2025.csv",
                sep=";",
                encoding="latin1",
                low_memory=False,
            ),
            dtype_mapping,
        )
    )
    df.to_csv(f"../data/raw/srag_20{y}.csv", index=False, date_format="%Y-%m-%d")
    print("Done.")

In [5]:
print("Merging raw data into one database")
merged_df = concat(DFS, ignore_index=True)
merged_df.to_csv(
    "../data/interim/srag_2019_2024.csv", index=False, date_format="%Y-%m-%d"
)
df_parquet = optimize_for_target_format(merged_df, "parquet")

for col in df_parquet.select_dtypes(["category"]).columns:
    if df_parquet[col].cat.categories.inferred_type not in ["string", "unicode"]:
        df_parquet[col] = df_parquet[col].astype("string").astype("category")

df_parquet.to_parquet(  # Não otimizando pq não limpei a base
    "../data/interim/srag_2019_2024.parquet", index=False, compression="snappy"
)
with sqlite3.connect("../data/interim/srag_2019_2024.db") as conn:
    optimize_for_target_format(merged_df, "sqlite").to_sql(
        "srag_cases",
        conn,
        if_exists="replace",
        index=False,
        dtype={
            col: TIMESTAMP
            for col, dtype in merged_df.dtypes.items()
            if dtype == "datetime64[ns]"
        },
    )

Merging raw data into one database


C:\Users\VDUART10\AppData\Local\Temp\ipykernel_47112\2390960348.py:3: DtypeWarning: Columns (17,92,94,106,108,115,117,118,119,121,122,123,129,144,146,149,155,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,181,185,186,189) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = read_csv("../data/interim/srag_2019_2024.csv")


## Usando NEWS API e salvando embeddings
criei uma unica coleção no qdrant para armazenar artigos de noticias sobre gripe, covid, sars e relacionados.
O fluxo é bem simples, apenas para alimentar a base de dados, num caso real teria q ser feito uma tratamento maior nas bases e aplicar criterios de escolha aos artigos como confiabilidade da fonte por exemplo. Segue a descrição:

1. dados são coletados usando a NEWs API para retorar artigos (você vai precisar criar uma key)
2. os artigos selecionados (tudo, não apliquei pesos ou discarte) são processados e salvos na coleção

In [ ]:
class NewsAPICollector:
    def __init__(self):
        self.BASE_URL = "https://newsapi.org/v2/everything"
        # isso devia estar em um vault ou secret manager, mas vou usar o .env pra simular
        self.API_KEY = os.getenv("NEWS_API_KEY")

    def fetch_articles(
        self,
        query: str,
        sort_by: str = "relevancy",  # Use 'relevancy' instead of 'popularity'
        days_back: int = 29,
    ) -> list[dict[str, str]]:
        """
        Fetch articles from News API for a specific query
        """
        to_date = datetime.now()
        from_date = to_date - timedelta(days=min(days_back, 29))  # Ensure max 29 days
        print(f"Fetching articles from {from_date} to {to_date} for query '{query}'")
        try:
            data: dict[str, Any] = (
                get(
                    self.BASE_URL,
                    params={
                        "q": query,
                        "from": from_date.strftime("%Y-%m-%d"),
                        "to": to_date.strftime("%Y-%m-%d"),
                        "sortBy": sort_by,  # 'relevancy', 'popularity', or 'publishedAt'
                        "apiKey": self.API_KEY,
                    },
                )
                .raise_for_status()
                .json()
            )

            if data["status"] == "ok":
                return data["articles"]
            else:
                print(f"API Error: {data.get('message', 'Unknown error')}")
                return []

        except HTTPStatusError as e:
            print(f"Request failed: {e}")
            raise e

    def preprocess_articles(
        self, articles: list[dict[str, str]]
    ) -> list[dict[str, str | dict[str, str]]]:
        """
        Clean and preprocess articles for RAG system
        """
        try:
            return [
                {
                    "id": f"news_{hash(article['url'])}",
                    # Combine title, description, and content
                    "text": f"{article.get('title', '')} {article.get('description', '')} {article.get('content', '')}".strip(),
                    "title": article.get("title", ""),
                    "url": article.get("url", ""),
                    "metadata": {
                        "source_type": "news_api",
                        "topic": "sars_cov",  # static topic for this use case
                        "url": article.get("url", ""),
                        "published_date": article.get("publishedAt", ""),
                        "source_name": article.get("source", {}).get("name", ""),
                        "image_url": article.get("urlToImage", ""),
                    },
                }
                for article in articles
                if article.get("content") or article["content"] != "[Removed]"
            ]
        except Exception as e:
            print(f"Error preprocessing articles: {e}")
            raise e

In [ ]:
class QdrantRAGSystem:
    def __init__(
        self,
        qdrant_url: str = "http://localhost:6333",
        collection_name: str = "sars_cov_news",
        embedding_model: str = "all-MiniLM-L6-v2",
    ):
        self.client = QdrantClient(url=qdrant_url)
        self.collection_name = collection_name
        self.encoder = SentenceTransformer(embedding_model)
        self.vector_size = self.encoder.get_sentence_embedding_dimension()
        self._create_collection()  # Create collection if it doesn't exist

    def _create_collection(self):
        """Create Qdrant collection for storing embeddings"""
        try:
            # Check if collection exists
            if self.collection_name not in [
                col.name for col in self.client.get_collections().collections
            ]:
                self.client.create_collection(
                    collection_name=self.collection_name,
                    vectors_config=VectorParams(
                        size=self.vector_size, distance=Distance.COSINE
                    ),
                )
                print(f"Created collection: {self.collection_name}")
            else:
                print(f"Collection {self.collection_name} already exists")

        except Exception as e:
            print(f"Error creating collection: {e}")

    def add_documents(self, documents: list[dict[str, str | dict[str, str]]]):
        """Add documents to Qdrant with embeddings"""
        points = [
            PointStruct(
                id=str(uuid.uuid4()),
                # Generate embedding for the text
                vector=self.encoder.encode(doc["text"]).tolist(),
                payload={
                    "text": doc["text"],
                    "title": doc["title"],
                    "url": doc["url"],
                    "metadata": doc["metadata"],
                },
            )
            for doc in documents
        ]  # Create points for Qdrant
        try:  # Batch upload to Qdrant
            self.client.upsert(collection_name=self.collection_name, points=points)
            print(f"Successfully added {len(points)} documents to Qdrant")
        except Exception as e:
            print(f"Error adding documents to Qdrant: {e}")

    def search_similar(self, query: str, limit: int = 5) -> list[dict]:
        """Search for similar documents using vector similarity"""
        try:
            return [
                {
                    "score": result.score,
                    "text": result.payload["text"],
                    "title": result.payload["title"],
                    "url": result.payload["url"],
                    "metadata": result.payload["metadata"],
                }
                for result in self.client.search(
                    collection_name=self.collection_name,
                    query_vector=self.encoder.encode(query).tolist(),
                    limit=limit,
                )
            ]
        except Exception as e:
            print(f"Error searching documents: {e}")
            return []

In [ ]:
class SARSCoVRAGPipeline:
    def __init__(self, qdrant_url: str = "http://localhost:6333"):
        self.news_collector = NewsAPICollector()
        self.rag_system = QdrantRAGSystem(qdrant_url=qdrant_url)

    def collect_and_index_news(self, query: str):
        """Complete pipeline to collect news and index in RAG system"""
        print("Fetching articles from News API...")
        if not (articles := self.news_collector.fetch_articles(query=query)):
            print("No articles found")
            return

        print(f"Found {len(articles)} articles. Preprocessing...")
        processed_docs = self.news_collector.preprocess_articles(articles)
        print(f"Processed {len(processed_docs)} articles. Adding to Qdrant...")
        self.rag_system.add_documents(processed_docs)
        print("Pipeline completed successfully!")

    def query_rag_system(self, user_query: str, num_results: int = 3):
        """Query the RAG system and return relevant information"""
        print(f"Searching for: {user_query}")
        if not (results := self.rag_system.search_similar(user_query, num_results)):
            print("No relevant articles found")
            return

        print(f"\nFound {len(results)} relevant articles:")
        print("=" * 50)

        for i, result in enumerate(results, 1):
            print(f"\n{i}. {result['title']}")
            print(f"   Relevance Score: {result['score']:.3f}")
            print(f"   Source: {result['metadata']['source_name']}")
            print(f"   URL: {result['url']}")
            print(f"   Preview: {result['text'][:200]}...")
            print("-" * 50)

In [ ]:
pipeline = SARSCoVRAGPipeline()

for query in ["covid", "sars", "flu", "influenza", "infectious", "viral"]:
    print(f"\n{'=' * 60}")
    pipeline.collect_and_index_news(query)

In [ ]:
pipeline.query_rag_system("What are the latest variants of SARS-CoV-2?", num_results=3)